<a href="https://colab.research.google.com/github/ashlearo/ML/blob/main/%D0%94%D0%97_3_%D0%95%D1%80%D0%BC%D0%B0%D0%BA%D0%BE%D0%B2_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Загружаем данные с каггла

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c sberbank-russian-housing-market #-p /content/gdrive/My\ Drive/kaggle/cancer

In [ ]:
!ls

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
warnings.filterwarnings(action='ignore')

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
#import catboost as cb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

##Разбиение данных на training sample и validation sample

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
print("train shape: ", train_df.shape)
print("test shape: ",test_df.shape)

In [ ]:
train_df.head()

##Анализ признаков и подготовка данных

In [ ]:
y = train_df['price_doc']

In [ ]:

f, ax = plt.subplots(figsize=(18, 12))
sns.distplot(y);

In [ ]:
fig = plt.figure(figsize = (10,6))
res = stats.probplot(y, plot=plt)

Линейный график не соответсвует графику стоимости, попробуем логарифм

In [ ]:
fig = plt.figure(figsize = (10,6))
res = stats.probplot(np.log1p(y), plot=plt)

In [ ]:
f, ax = plt.subplots(figsize=(18,12))
highcorr_vars = (train_df.corr().price_doc.sort_values(ascending=False)).index  #[:10]
sns.heatmap(train_df[highcorr_vars].corr(), annot=True)
plt.show()

In [ ]:
plt.subplots(figsize=(10, 8))
outlier = train_df['full_sq'].sort_values(ascending=False)[:2].index
sns.scatterplot(x=train_df['full_sq'], y=y)
sns.scatterplot(x=train_df.iloc[outlier]['full_sq'], y=train_df.iloc[outlier]['price_doc'], color='r', s=100, alpha=.6)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
outlier = train_df['life_sq'].sort_values(ascending=False)[:1].index
sns.scatterplot(x = train_df['life_sq'], y = y)
sns.scatterplot(x=train_df.iloc[outlier]['life_sq'], y=train_df.iloc[outlier]['price_doc'], color='r', s=100, alpha=.6)
plt.ylabel('price_doc', fontsize=13)
plt.xlabel('life_sq', fontsize=13)
plt.show()

Нанесем на одном графике распределение стоимости от жилой и общей площади.
результат - оба этих параметра сильно коррелиют между собой

In [ ]:
plt.subplots(figsize=(10, 8))
outlier = train_df['full_sq'].sort_values(ascending=False)[:2].index
sns.scatterplot(x=train_df['full_sq'], y = y, color='c')
sns.scatterplot(x = train_df['life_sq'], y = y, color='g')
# sns.scatterplot(x=train_df.iloc[outlier]['full_sq'], y=train_df.iloc[outlier]['price_doc'], color='r', s=100, alpha=.6)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
outlier = train_df['sub_area'].sort_values(ascending=False)[:1].index
sns.scatterplot(x = train_df['sub_area'], y = y)
sns.scatterplot(x=train_df.iloc[outlier]['sub_area'], y=train_df.iloc[outlier]['price_doc'], color='r', s=100, alpha=.6)
plt.ylabel('price_doc', fontsize=13)
plt.xlabel('sub_area', fontsize=13)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
outlier = train_df['green_zone_part'].sort_values(ascending=False)[:1].index
sns.scatterplot(x = train_df['green_zone_part'], y = y)
sns.scatterplot(x=train_df.iloc[outlier]['green_zone_part'], y=train_df.iloc[outlier]['price_doc'], color='r', s=100, alpha=.6)
plt.ylabel('price_doc', fontsize=13)
plt.xlabel('green_zone_part', fontsize=13)
plt.show()

Гистограмма плотности распределения цены объекта

In [ ]:

plt.rcParams['text.color'] = 'black'
f, ax = plt.subplots(figsize=(4, 4))
sns.distplot(y)
# sns.distplot(train_df.price_doc, fit=norm, color='r')
plt.title('Распределение цены объекта', size=15)
plt.xlabel('Цена', size=10)
plt.ylabel('Плотность', size=10)
plt.show()

Гистограмма плотности распределения логарифма цены объекта, ближе к плотности нормального распределения, только имеются пара выбросов при низкой стоимости объектов, их бы надо найти и удалить из дата сета(либо заменить на средние значения)

In [ ]:
f, ax = plt.subplots(figsize=(4, 4))
sns.distplot(np.log1p(y))
plt.title('Логарифм распределения', size=15)
plt.xlabel('Цена', size=10)
plt.ylabel('Плотность', size=10)
plt.show()

код ниже 3 ячейки не исользую

In [ ]:
train_df.drop(train_df['GrLivArea'].sort_values(ascending=False)[:2].index, axis=0, inplace=True)
train_df.drop(train_df['1stFlrSF'].sort_values(ascending=False)[:1].index, axis=0, inplace=True)
train_df.drop(train_df['TotalBsmtSF'].sort_values(ascending=False)[:1].index, axis=0, inplace=True)

In [ ]:
train_id = train_df.Id
test_id = test_df.Id
train_df.drop(['Id'], axis=1, inplace=True)
test_df.drop(['Id'], axis=1, inplace=True)

In [ ]:
train_df = train_df.drop(['Alley','PoolQC','MiscFeature'],axis=1)
test_df = test_df.drop(['Alley','PoolQC','MiscFeature'],axis=1)

In [ ]:
train_size = train_df.shape[0]
y_train = train_df['price_doc'].values
all_data = pd.concat((train_df, test_df), axis=0).reset_index(drop=True)
all_data.drop(['price_doc'], axis=1, inplace=True)

In [ ]:
train_df.shape, test_df.shape, all_data.shape, y_train.shape

((30471, 292), (7662, 291), (38133, 291), (30471,))

In [ ]:
all_data.isnull().sum().sort_values(ascending=False).head(20)

In [ ]:
f, ax = plt.subplots(6, 6, figsize=(25, 20))
for i, c in enumerate(continuous_vars):
    sns.distplot(all_data[c], fit=norm, ax=ax[i//6, i%6])

*Заполним пропуски и применим OneHotEncoding*


---

In [ ]:
cat_features_mask = (all_data.dtypes == "object").values

In [ ]:
# для вещественнозначных признаков заполним пропуски медианой
X_real = all_data[all_data.columns[~cat_features_mask]]
mis_replacer = SimpleImputer(strategy="median")
X_no_mis_real = pd.DataFrame(data=mis_replacer.fit_transform(X_real), columns=X_real.columns)
# для категориальных - "None"
X_cat = all_data[all_data.columns[cat_features_mask]].fillna("None")
all_data = pd.concat([X_no_mis_real, X_cat], axis=1)

In [ ]:
all_data = pd.get_dummies(all_data)

*Разделим на train и test*


---

In [ ]:
y_train = np.log1p(y_train)

In [ ]:
X_train, X_test = all_data.iloc[:train_size, :], all_data.iloc[train_size:, :]

In [ ]:
X_train.shape, X_test.shape, y_train.shape

##Обучение модели

In [ ]:
cb_model = cb.CatBoostRegressor(
    depth = 4,
    l2_leaf_reg = 1,
    learning_rate = 0.03)

In [ ]:
model_xgb = xgb.XGBRegressor(
         objective = 'reg:squarederror',
         colsample_bytree = 0.5,
         learning_rate = 0.1,
         max_depth = 3,
         min_child_weight = 1,
         n_estimators = 500,
         subsample = 0.7)

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05,
                              n_estimators=720,
                              max_bin = 55,
                              bagging_fraction = 0.8,
                              bagging_freq = 5,
                              feature_fraction = 0.25,
                              feature_fraction_seed=9,
                              bagging_seed=9,
                              min_data_in_leaf =6,
                              min_sum_hessian_in_leaf = 11)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#XGBoost hyper-parameter tuning
def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
    }

    model_xgb = xgb.XGBRegressor()

    gsearch2 = GridSearchCV(estimator = model_xgb,
                           param_grid = param_tuning,
                           scoring = 'neg_root_mean_squared_error',
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gsearch2.fit(X_train,y_train)

    return gsearch2.best_params_

In [ ]:
hyperParameterTuning(X_train, y_train)

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

##install catboost

In [ ]:
!pip install catboost

##Обучение CatBoost

In [ ]:
cb_model.fit(X_train, y_train)

In [ ]:
cb_train_pred = cb_model.predict(X_train)
cb_pred = np.expm1(cb_model.predict(X_test))
print(rmse(y_train, cb_train_pred))
print(r2_score(y_train, cb_train_pred))

##Обучение XGBRegressor

In [ ]:
model_xgb.fit(X_train, y_train)
xgb_train_pred = model_xgb.predict(X_train)
xgb_pred = np.expm1(model_xgb.predict(X_test))
print(rmsle(y_train, xgb_train_pred))
print(r2_score(y_train, xgb_train_pred))

##Обучение LGBMRegressor

In [ ]:
model_lgb.fit(X_train, y_train)
lgb_train_pred = model_lgb.predict(X_train)
lgb_pred = np.expm1(model_lgb.predict(X_test.values))
print(rmsle(y_train, lgb_train_pred))
print(r2_score(y_train, lgb_train_pred))

##Подготовка результатов на Kaggle


In [ ]:
predictions = (cb_pred + xgb_pred + lgb_pred) / 3

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_id
sub['SalePrice'] = predictions
sub.to_csv('submission_0311_8.csv',index=False)

In [ ]:
sub